<h1><center>Лабораторна робота 7.</center></h1>
<h2><center>Аналіз активності людини за відкритими даними мобільних телефонів</center></h2>

**Виконав:** Прізвище І.П.

**Варіант:** №__

<a class="anchor" id="lab-7"></a>

## Зміст

- [7.1. Завантаження та підготовка даних](#lab-7.1)
- [7.2. Застосування методу головних компонент](#lab-7.2)  
- [7.3. Застосування *k*-means](#lab-7.3)
- [7.4. Агломератична кластеризація](#lab-7.4)
- [7.5. Застосування класифікатора після кластеризації](#lab-7.5)
- [7.6. Кластеризація із методом головних компонент](#lab-7.6)

<a class="anchor" id="lab-7.1"></a>

## <span style="color:blue; font-size:1.2em;">7.1. Завантаження та підготовка даних</span>

[Повернутися до змісту](#lab-7)

У цій лабораторній робота Вам потрібно розібратися з тим, як працюють методи пониження розмірності й кластеризації даних. Крім того, ще раз попрактикуємося із задачею класифікації.

Тут ми працюватимемо з набором даних [Samsung Human Activity Recognition](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones). Завантажте дані [звідси](https://drive.google.com/file/d/14RukQ0ylM2GCdViUHBBjZ2imCaYcjlux/view?usp=sharing).

Дані взято з акселерометрів і гіроскопів мобільних телефонів Samsung Galaxy S3 (докладніше щодо ознак – за посиланням на UCI вище), також відомий вид активності людини з телефоном в кишені – чи ходила людина, стояла, лежала, сиділа або йшла вгору/вниз сходами.

Спершу представимо, що вид активності нам невідомий, і спробуємо кластеризувати людей лише на основі наявних свідчень. Далі розв'яжемо завдання визначення виду фізичної активності саме як задачу класифікації.

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook

%matplotlib inline
from matplotlib import pyplot as plt
plt.style.use(['seaborn-darkgrid'])
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.family'] = 'DejaVu Sans'

from sklearn import metrics
from sklearn.cluster import MiniBatchKMeans, AgglomerativeClustering, SpectralClustering
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

RANDOM_STATE = 17

In [4]:
X_train = np.loadtxt("samsung_HAR/samsung_train.txt")
y_train = np.loadtxt("samsung_HAR/samsung_train_labels.txt").astype(int)

X_test = np.loadtxt("samsung_HAR/samsung_test.txt")
y_test = np.loadtxt("samsung_HAR/samsung_test_labels.txt").astype(int)

In [5]:
# Альтернативне завантаження даних.
# Розкоментувати цей код, якщо
# завантаження тексктових даних вище не працює
'''
X_train_url = "https://raw.githubusercontent.com/radiukpavlo/applied-math-packages/main/02_assignments/amp_lab-07_pca-kmeans/samsung_HAR/samsung_train.txt"
y_train_url = "https://raw.githubusercontent.com/radiukpavlo/applied-math-packages/main/02_assignments/amp_lab-07_pca-kmeans/samsung_HAR/samsung_train_labels.txt"

X_test_url = "https://raw.githubusercontent.com/radiukpavlo/applied-math-packages/main/02_assignments/amp_lab-07_pca-kmeans/samsung_HAR/samsung_test.txt"
y_test_url = "https://raw.githubusercontent.com/radiukpavlo/applied-math-packages/main/02_assignments/amp_lab-07_pca-kmeans/samsung_HAR/samsung_test_labels.txt"

X_train = np.loadtxt(X_train_url)
y_train = np.loadtxt(y_train_url)
X_test = np.loadtxt(X_test_url)
y_test = np.loadtxt(y_test_url)
'''

'\nX_train_url = "https://raw.githubusercontent.com/radiukpavlo/applied-math-packages/main/02_assignments/amp_lab-07_pca-kmeans/samsung_HAR/samsung_train.txt"\ny_train_url = "https://raw.githubusercontent.com/radiukpavlo/applied-math-packages/main/02_assignments/amp_lab-07_pca-kmeans/samsung_HAR/samsung_train_labels.txt"\n\nX_test_url = "https://raw.githubusercontent.com/radiukpavlo/applied-math-packages/main/02_assignments/amp_lab-07_pca-kmeans/samsung_HAR/samsung_test.txt"\ny_test_url = "https://raw.githubusercontent.com/radiukpavlo/applied-math-packages/main/02_assignments/amp_lab-07_pca-kmeans/samsung_HAR/samsung_test_labels.txt"\n\nX_train = np.loadtxt(X_train_url)\ny_train = np.loadtxt(y_train_url)\nX_test = np.loadtxt(X_test_url)\ny_test = np.loadtxt(y_test_url)\n'

In [6]:
# Перевіримо розмірності
assert(X_train.shape == (7352, 561) and y_train.shape == (7352,))
assert(X_test.shape == (2947, 561) and y_test.shape == (2947,))

Для кластеризації нам не потрібен вектор відповідей, тому будемо працювати з об'єднанням навчальної та тестової вибірок. Об'єднаємо `X_train` з `X_test` та `y_train` з `y_test`.

In [7]:
X = np.vstack([X_train, X_test])
y = np.hstack([y_train, y_test])

Визначимо кількість унікальних значень міток цільового класу.

In [8]:
np.unique(y)

array([1, 2, 3, 4, 5, 6])

In [9]:
n_classes = np.unique(y).size

Ці мітки [відповідають](https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.names) таким значенням:
- 1 – ходьба;
- 2 – підйом вгору сходами;
- 3 – спуску вниз сходами;
- 4 – сидіння;
- 5 – стояння;
- 6 – лежання.

Виконаємо масштабування вибірки за допомогою `StandardScaler` з параметрами за замовчуванням:

In [10]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

------------------------

<a class="anchor" id="lab-7.2"></a>

## <span style="color:blue; font-size:1.2em;">7.2. Застосування методу головних компонент</span>

[Повернутися до змісту](#lab-7)

Виконайте зниження розмірності даних за допомогою PCA. Залишіть таку кількість компонент, щоби пояснити як мінімум 90% дисперсії оброблених (масштабованих) даних. Використовуйте масштабовану вибірку й зафіксуйте random_state (константа RANDOM_STATE).

In [ ]:
# Ваш код тут
pca = 
X_pca = 

In [ ]:
# Переглядаємо кількість компонент
X_pca.shape

### <span style="color:red; font-size:1.5em;">Завдання 1</span>


**Для варіантів 1-5:**

Виконайте ще одне пониження розмірності даних, але для пояснення 100% дисперсії. Порівняйте результати з точки зору необхідної кількості компонент.

**Для варіантів 6-10:**

Яку мінімальну кількість головних компонент потрібно виділити, щоб пояснити 90% дисперсії початкових (масштабованих) даних? 

**Для варіантів 11-15:**

Скільки відсотків дисперсії припадає на першу головну компоненту? Використайте метод `explained_variance_ratio_`. Результат округліть до сотих.

**Для варіантів 16-20:**

Виведіть номер головної компоненти, на яку припадає найбільший % дисперсії (для 90% дисперсії). Використайте метод `explained_variance_ratio_`.

### <span style="color:red; font-size:1.5em;">Додаткове завдання</span>

**Для всіх варіантів:**

Відобразіть дані в проекції на перші дві головні компоненти. Для візуалізації використайте `plt.scatter()`. 

Якщо візуалізацію виконано правильно, Ви побачите скільки-то кластерів, майже ідеально відокремлених один від одного. Які види активності входять в ці кластери?

------------------------

<a class="anchor" id="lab-7.3"></a>

## <span style="color:blue; font-size:1.2em;">7.3. Застосування *k*-means</span>

[Повернутися до змісту](#lab-7)

Виконайте кластеризацію даних методом `MiniBatchKmeans`, навчивши модель за даними зі зниженою за рахунок PCA розмірністю. Тут потрібно шукати саме 6 кластерів, але загалом у задачах без вчителя ми не знаємо, яку кількість кластерів треба шукати.

Параметри:

- **n_clusters** = n_classes (кількість унікальних міток цільового класу);
- **n_init** = 100;
- **random_state** = RANDOM_STATE (для відтворюваності результату)

Остальные параметры со значениями по умолчанию.

In [ ]:
kmeans = # Ваш код тут

kmeans.fit(X_pca)

cluster_labels = # Ваш код тут

Перегляньте відповідність між мітками кластерів і початковими мітками класів, а також гляньте на те, які види активностей алгоритм `MiniBatchKmeans` плутає.

In [ ]:
tab = crosstab(y, cluster_labels, margins=True)
tab.index = ['ходьба', 'підйом вгору сходами', 
             'спуск вниз сходами', 'сидіння', 'стояння', 'лежання', 'всі']
tab.columns = ['cluster' + str(i + 1) for i in range(6)] + ['всі']
tab

Бачимо, що кожному класу (тобто кожній активності) відповідають кілька кластерів. Давайте подивимося на максимальну частку об'єктів в класі, що відносяться до якогось одного кластеру. Ця частка буде простокою метрикою, що характеризує наскільки легко клас відділяється від інших під час кластеризування.

Приклад: якщо для класу "спуск сходами", в якому 1406 об'єктів, розподіл кластерів є таким:
 - кластер 1 – 900;
 - кластер 3 – 500;
 - кластер 6 – 6,
 
то така частка буде 900 / 1406 $\approx$ 0.64.

Зазвичай, *k*-means не надто добре відрізняє тільки активності одну від одної. Тому використаємо метод ліктя, щоби вибрати оптимальну кількість кластерів. Параметри алгоритму й дані використовуємо ті ж, що і раніше, міняємо лише `n_clusters`.

In [ ]:
inertia = []
for k in tqdm_notebook(range(1, n_classes + 1)):
    kmeans = MiniBatchKmeans(n_clusters=k, n_init=100, 
                    random_state=RANDOM_STATE, n_jobs=1).fit(X_pca)
    inertia.append(np.sqrt(kmeans.inertia_))

plt.plot(range(1, 7), inertia, marker='s');

### <span style="color:red; font-size:1.5em;">Завдання 2</span>


**Для варіантів 1-5:**

Яка кількість кластерів є оптимальною? Для відповіді на це запитання використайте метод ліктя, а саме закодуйте $D(k)$ (аналогічно до прикладу з лекції).

**Для варіантів 6-10:**

 Чи правда, що краще обрати 3 кластери для розбиття всієї відмасштабованої вибірки? Для відповіді на це запитання використайте метод ліктя, а саме закодуйте $D(k)$ (аналогічно до прикладу з лекції).

**Для варіантів 11-15:**

Обрахуйте, який вид активності відокремився від інших найкраще за метрикою частки об'єктів в класі (описано вище)?

**Для варіантів 16-20:**

Візуалізуйте дані в проекції на перші дві головні компоненти. Зображіть точки різними кольорами, відповідно до отриманих міток кластерів (`cluster_labels`). Використайте `plt.scatter()`.

------------------------

<a class="anchor" id="lab-7.4"></a>

## <span style="color:blue; font-size:1.2em;">7.4. Агломератична кластеризація</span>

[Повернутися до змісту](#lab-7)

Спробуємо ще один алгоритм, що входить до групи ієрархічної кластеризації – агломеративну кластеризацію [`AgglomerativeClustering()`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html).

In [ ]:
ag = # Ваш код тут

Обрахуйте Adjusted Rand Index (ARI) для розбиття на два кластери за [`AgglomerativeClustering()`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) та [`MiniBatchKmeans()`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html) з параметрами із пункту 7.3.

Відповідно до знайденого значення ARI, дайте відповіді на запитання нижче.

### <span style="color:red; font-size:1.5em;">Завдання 3</span>


**Для варіантів 1-5:**

Порівняйте значення ARI, знайденого для `AgglomerativeClustering()` та `MiniBatchKmeans()`. Поясніть, чому ці значення різні.

**Для варіантів 6-10:**

Відповідно до виконаних обрахунків, що впливає на значення ARI? Присвоєні мітки чи розбиття об'єктів на кластери? Наведіть коротке пояснення.

**Для варіантів 11-15:**

Чи правда, що для нашого прикладу `AgglomerativeClustering()` впорався з кластеризацією гірше, ніж `MiniBatchKmeans()`? Чому?

**Для варіантів 16-20:**

Чи правда, що у разі випадкового розбиття на кластери ARI буде близький до нуля. Чому?

------------------------

<a class="anchor" id="lab-7.5"></a>

## <span style="color:blue; font-size:1.2em;">7.5. Застосування класифікатора після кластеризації</span>

[Повернутися до змісту](#lab-7)

Можна помітити, що завдання не надто добре розв'язується саме як задача кластеризації, якщо виділяти лише кілька кластерів (>2). Спробуємо тепер розв'яжемо задачу класифікації; пам'ятаємо, що дані у нас розмічені.

Для класифікації використайте метод опорних векторів – клас `sklearn.svm.LinearSVC`. В межах курсу ми не розглядали відокремлено цей алгоритм, проте він вкрай відомий, почитати про нього можна, наприклад, в документації scikit-learn під назвою [Support Vector Machines](https://scikit-learn.org/stable/modules/svm.html#). 

Налаштуйте для `LinearSVC` гіперпараметр `C` за допомогою `GridSearchCV` (cv=3). Виконайте навчання нового `StandardScaler` за навчальною вибіркою (з усіма початковими ознаками), застосуйте масштабування до тестової вибірки.

In [ ]:
scaler = StandardScaler()
X_train_scaled = # Ваш код тут
X_test_scaled = # Ваш код тут

In [ ]:
svc = LinearSVC(random_state=RANDOM_STATE)
svc_params = {'C': [0.001, 0.01, 0.1, 1, 10]}

In [ ]:
%%time
# Ваш код тут
best_svc = GridSearchCV(...)
best_svc.fit(X_train_scaled, y_train);

In [ ]:
# Ваш код тут
# Виведіть значення гіперпараметра С

Виконайте прогноз за тестовим набором даних:

In [ ]:
y_predicted = # Ваш код тут

Вивидимо результат розподілення класів у таблиці:

In [ ]:
tab = pd.crosstab(y_test, y_predicted, margins=True)
tab.index = ['ходьба', 'підйом вгору сходами', 'спуску вниз сходами', 
             'сидіння', 'стояння', 'лежання', 'всі']
tab.columns = tab.index
tab

### <span style="color:red; font-size:1.5em;">Завдання 4</span>

Для відповіді на запитання нижче використайте `metrics.classification_report()`:

**Для варіантів 1-5:**

Обрахуйте частку правильних відповідей (accuracy) класифікатора SVM? Виведіть значення відповідної метрики за допомогою коду.

**Для варіантів 6-10:**

Який вид активності SVM визначає найгірше за метрикою влучності (precision)? Виведіть назву відповідної ознаки (вид активності) та значення відповідної метрики за допомогою коду.

**Для варіантів 11-15:**

Який вид активності SVM визначає найгірше за метрикою повноти (recall)? Виведіть назву відповідної ознаки (вид активності) та значення відповідної метрики за допомогою коду.

**Для варіантів 16-20:**

Який вид активності SVM визначає найгірше за метрикою $F$-міри (f1-score)? Виведіть назву відповідної ознаки (вид активності) та значення відповідної метрики за допомогою коду.

------------------------

<a class="anchor" id="lab-7.6"></a>

## <span style="color:blue; font-size:1.2em;">7.6. Кластеризація із методом головних компонент</span>

[Повернутися до змісту](#lab-7)

Виконайте навчання класифікатора, повторивши всі попередні дії (завдання 4) з додаванням PCA:

- Використовуйте вибірки `X_train_scaled` і `X_test_scaled`.
- Навчіть той же PCA, що й раніше за масштабованою навчальною вибіркою; застосуйте перетворення до тестової вибірки.
- Налаштуйте гіперпараметр `C` на крос-валідації за навчальною вибіркою з PCA-перетворенням.

In [ ]:
scaler = StandardScaler()
X_train_scaled = # Ваш код тут
X_test_scaled = # Ваш код тут

pca = PCA(...)
X_train_pca = # Ваш код тут
X_test_pca = # Ваш код тут

In [ ]:
svc = LinearSVC(random_state=RANDOM_STATE)
svc_params = {'C': [0.001, 0.01, 0.1, 1, 10]}

In [ ]:
%%time
best_svc_pca = # Ваш код тут
best_svc_pca.fit(...);

### <span style="color:red; font-size:1.5em;">Завдання 5</span>

Обрахуйте метрики за класифікатором SVM та з використанням PCA:

**Для варіантів 1-5:**

Яка різниця між кращою якістю (accuracy) на крос-валідації за використанням всіх 561 початкових ознак і за  використання методу головних компонент? Заокругліть значення метрики до сотих.

**Для варіантів 6-10:**

Порівняйте значення влучності (precision), отриманого за всіма ознаками (завдання 4) і з використанням PCA. Вкажіть переваги/недоліки застосування PCA до навчальної вибірки.

**Для варіантів 11-15:**

Порівняйте значення повноти (recall), отриманого за всіма ознаками (завдання 4) і з використанням PCA. Вкажіть переваги/недоліки застосування PCA до навчальної вибірки.

**Для варіантів 16-20:**

Порівняйте значення $F$-міри (f1-score), отриманої за всіма ознаками (завдання 4) і з використанням PCA. Вкажіть переваги/недоліки застосування PCA до навчальної вибірки.